In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer
import torch

model_dest = "tollefj/nordavind-instruct-7b-v0"
datatype = torch.float16
tokens = 128
model_temp = 0.75

model = AutoModelForCausalLM.from_pretrained(model_dest, torch_dtype=datatype, use_cache=True, device_map="auto")
tok = AutoTokenizer.from_pretrained(model_dest)

In [ ]:
streamer = TextStreamer(tok, skip_prompt=True)
pipe = pipeline("text-generation", 
                model=model,
                tokenizer=tok,
                torch_dtype=datatype,
                streamer=streamer,
                eos_token_id=tok.eos_token_id,
                device_map="auto")

In [ ]:
tok.encode("<|im_end|>")

In [ ]:
system_message = """Du er Nordavind, en hjelpsom assistent. Du skal svare på spørsmål og hjelpe brukere med å finne nødvendig informasjon."""

def make_prompt(instruction):
    chat = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": instruction},
    ]
    return pipe.tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )
    
def pred(instruction, tokens=100, temp=model_temp):
    prompt = make_prompt(instruction)
    outputs = pipe(
        prompt,
        max_new_tokens=tokens,
        do_sample=True,
        temperature=temp,
        top_k=50,
        top_p=0.1,
        no_repeat_ngram_size=3,
        # eos_token_id=pipe.tokenizer.eos_token_id,
        # pad_token_id=pipe.tokenizer.pad_token_id,
    )

pred("Hvilke steg trengs for å lage en maskinlæringsmodell? Svar med en punktliste")

In [ ]:
pred("Nevn tre ting som er typisk norsk")

In [ ]:
instruction = "Produkt type kan være en av følgende kategorier: 'Annet/Ikke_oppgitt', 'Mobil', 'Bredbånd-mobilt', 'Bredbånd', 'Tjenester/Apper', 'Forsikring', 'E-post', 'TV' eller 'Telefoni'. Angi produkt type som det tales om her:\n"

phonecall = """hei du taler med _navn_. hei du jeg har problemer med dekning på telefonen min på hytta, det er ikke så bra for jeg skal gjerne være i kontakt med mine kone fordi hun er på sykehus. Ser den det er ikke så bra kan du gi meg personnumemret ditt? ja det er _anonymisert_. takk da skal jeg ta en titt, ser du har CallMi abo med 1 gigabait. Stemmer det. Ser at du har brukt op all data og derfor har din hastighet blitt redusert kraftig. Hva i helsike det kan ikke passe jeg har nesten ikke gjort noe på telefonem min. Skal jeg øke abonnementet ditt slik du får mere data denne måned? det koster 99 kroner. Ja bare gjør det da. Hei. Hei.
"""
user_prompt = instruction+phonecall

pred(user_prompt)